
$$
x_{ij} = \left\{
    \begin{array}\\
        1 & \mbox{if we place a queen in row i and column j} \\
        0 & \mbox{otherwise} \\
    \end{array}
\right. \;\;\;\;\;\; i,j = 1...n
$$

Place n queens on the board.
$$\sum_{i=1}^n\sum_{j=1}^nx_{ij} = n $$

Only one queen is allowed per row:

$$\sum_{j=1}^nx_{ij} \leq 1 \;\;\;\;\forall \; i = 1...n $$

Only one queen is allowed per column:

$$\sum_{i=1}^nx_{ij} \leq 1 \;\;\;\;\forall \; j = 1...n $$

Descending Diagonals

$$\sum_{i=1\\{i-j=k}}^n\sum_{j=1}^nx_{ij} \leq 1 \;\;\;\;\forall \; k \in \{1-n,...,n-1\}$$  

Ascending Diagonals

$$\sum_{i=1\\{i+j=l}}^n\sum_{j=1}^nx_{ij} \leq 1 \;\;\;\;\forall \; l \in \{2,...,2n\}$$  

In [20]:
from gurobipy import *

n=8

x={}

model = Model()

for i in range(n):
    for j in range(n):
        x[i+1,j+1] = model.addVar(vtype=GRB.BINARY, obj=1, name="x_{}{}".format(i+1,j+1))


model.update()

model.addConstr(quicksum(x[i+1,j+1] for i in range(n) for j in range(n)) == n)

for i in range(n):
    model.addConstr(quicksum(x[i+1,j+1] for j in range(n)) <= 1)

for j in range(n):
    model.addConstr(quicksum(x[i+1,j+1] for i in range(n)) <= 1)

for k in range(1-n, n):
    model.addConstr(quicksum(x[i+1,j+1] for i in range(n) for j in range(n) if i-j == k) <= 1)

for l in range(0, 2*n-1):
    model.addConstr(quicksum(x[i+1,j+1] for i in range(n) for j in range(n) if i+j == l) <= 1)


# Gurobi Settings
model.params.LogToConsole = 0         # Quiet Gurobi
model.params.PoolSearchMode = 2       # Force Gurobi to search for multiple optimal solutions
model.params.PoolSolutions = 10**8    # Store max 10**8 Solutions in the solution pool
model.params.PoolGap = 0              # All Solutions in the pool have 0 % Gap = optimal solutions
model.optimize()


print(f"For n = {n} there are {model.SolCount} possible solutions")

if model.status == GRB.status.OPTIMAL:
    model.write("Queens.lp")
    model.write("Queens.sol")
elif model.status == GRB.status.INFEASIBLE:
    model.computeIIS() # IIS tells you which constraints, when removed, makes the model feasible
    model.write('inf.ilp')

For n = 8 there are 92 possible solutions
